### 1. Settings

In [1]:
# DARTS Library for Forecasting

import darts
from darts import TimeSeries
from darts.utils.timeseries_generation import gaussian_timeseries, linear_timeseries, sine_timeseries
from darts.models import LightGBMModel, CatBoostModel, Prophet, RNNModel, TFTModel, NaiveSeasonal, ExponentialSmoothing, NHiTSModel
from darts.metrics import mape, smape, rmse, rmsle
import sklearn.preprocessing
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import MissingValuesFiller, Scaler, InvertibleMapper
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis, plot_hist
from darts.utils.likelihood_models import QuantileRegression
from darts.utils.missing_values import fill_missing_values
from darts.models.filtering.moving_average_filter import MovingAverageFilter

# optuna 
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
)

# pytorch
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# data processing
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc

%matplotlib inline
torch.manual_seed(1); np.random.seed(1)  

c:\Users\Younkyung Lee\anaconda3\envs\KerasNN\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Build a Preprocessing Script

#### 2-1. Utility Functions

In [2]:
# Function to load data from CSV files
def load_data(filepaths):
    return {name: pd.read_csv(path) for name, path in filepaths.items()}

# Function to create TimeSeries objects
def create_time_series(df, time_col, group_cols, value_col, static_cols, freq='D'):
    return TimeSeries.from_group_dataframe(df, time_col=time_col, group_cols=group_cols, value_cols=value_col, static_cols=static_cols, fill_missing_dates=True, freq=freq)

# Function to apply transformations to target variable
def transform_time_series(time_series, transformers, type):
    pipeline = Pipeline(transformers)
    if type == 'nested':
        result = [pipeline.fit_transform(ts) for ts in time_series]
    elif type == 'single':
        result = pipeline.fit_transform(time_series)
    return result, pipeline

#### 2-2. Load and Process Data

In [3]:
## Load Data
filepaths = {
    'train': 'data/train.csv',
    'test': 'data/test.csv',
    'holidays_events': 'data/holidays_events.csv',
    'oil': 'data/oil.csv',
    'stores': 'data/stores.csv',
    'transactions': 'data/transactions.csv',
    'sample_submission': 'data/sample_submission.csv'
}
dataframes = load_data(filepaths)

df_train = dataframes['train']
df_test = dataframes['test']
df_stores = dataframes['stores']
df_holiday = dataframes['holidays_events']
df_oil = dataframes['oil']
df_transactions = dataframes['transactions']

family_list = df_train['family'].unique()
store_list = df_stores['store_nbr'].unique()

## Merge train + store data 
train_merged = pd.merge(dataframes['train'], dataframes['stores'], on = 'store_nbr')
train_merged = train_merged.sort_values(["store_nbr", "family", "date"])
train_merged = train_merged.astype({"store_nbr": 'str', "family": 'str', "city": 'str', "state": 'str', "type": 'str', "cluster": 'str'})

## Test Data
df_test_dropped = dataframes['test'].drop(['onpromotion'], axis=1)
df_test_sorted = df_test_dropped.sort_values(by=['store_nbr', 'family'])

#### 2-3. Target: Sales

In [4]:
## Create Time Series Objects by Product Family X Store Number
family_dict = {family: create_time_series(train_merged[train_merged['family'] == family], 
                                             'date', 
                                             ["store_nbr", "family"], 
                                             'sales', 
                                             ["city", "state", "type", "cluster"]) 
                                             for family in train_merged['family'].unique()}

## Transform Sales for each Product Family X Store Number 
transformers = [
    MissingValuesFiller(),
    StaticCovariatesTransformer(sklearn.preprocessing.OneHotEncoder()),
    InvertibleMapper(np.log1p, np.expm1),
    Scaler()
]

family_trans_dict = {}
family_pipeline_dict = {}

for key in family_dict:
    train_pipeline = Pipeline(transformers)
    transformed =train_pipeline.fit_transform(family_dict[key])
    family_trans_dict[key] = transformed
    family_pipeline_dict[key] = train_pipeline

## Create Moving Average by Product Family X Store Number
sales_moving_average_7 = MovingAverageFilter(window=7)
sales_moving_average_28 = MovingAverageFilter(window=28)

sales_ma_dict = {}

for key in family_trans_dict:
  sales_mas_family = []
  
  for ts in family_trans_dict[key]:
    ma_7 = sales_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())  
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="sales_ma_7")

    ma_28 = sales_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())  
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="sales_ma_28")

    mas = ma_7.stack(ma_28)
    sales_mas_family.append(mas)
  
  sales_ma_dict[key] = sales_mas_family

#-- [dict] family_dict, family_trans_dict, sales_ma_dict
#-- [keys (33)] product family [values (54)] dart time series object for each store

In [87]:
# Create TimeSeries objects (Darts) 1782

list_of_TS = TimeSeries.from_group_dataframe(
                                train_merged,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                static_cols=["city","state","type","cluster"], # also extract these additional columns as static covariates
                                value_cols="sales", # target variable
                                fill_missing_dates=True,
                                freq='D')
for ts in list_of_TS:
            ts = ts.astype(np.float32)

list_of_TS = sorted(list_of_TS, key=lambda ts: int(ts.static_covariates_values()[0,0]))

# Transform the Sales Data

train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") #OneHotEncoder would be better but takes longer
log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])
     
training_transformed = train_pipeline.fit_transform(list_of_TS)


#### 2-4. General Feature I: Oil

In [5]:
# Interpolate missing values in oil dataframe
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].astype(np.float32)
df_oil.interpolate(method='linear', inplace=True)

# Convert pandas dataframe to timeseries
oil = TimeSeries.from_dataframe(df_oil, time_col='date', value_cols=['dcoilwtico'], freq='D')
oil = oil.astype(np.float32)

# Transform Oil Timeseries
oil_transformers = [MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler"), 
                    Scaler(verbose=False, n_jobs=-1, name="Scaler")]
oil_pipeline = Pipeline(oil_transformers)
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price
oil_moving_average_7 = MovingAverageFilter(window=7)
oil_moving_average_28 = MovingAverageFilter(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="oil_ma_7")
ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="oil_ma_28")
oil_moving_averages = ma_7.stack(ma_28)

#-- [dart ts array] oil, oil_transformed, oil_moving_averages

C:\Users\Younkyung Lee\AppData\Local\Temp\ipykernel_15292\1519470149.py:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_oil.interpolate(method='linear', inplace=True)


#### 2-5. General Feature II: Date 

In [6]:
## Create and Transform Time based features
full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')

year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
day = datetime_attribute_timeseries(time_index = full_time_period, attribute="day")
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofyear")
weekday = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
timesteps = TimeSeries.from_times_and_values(times=full_time_period,
                                             values=np.arange(len(full_time_period)),
                                             columns=["linear_increase"])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

# Transform
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

In [16]:
## general covariates: time(date features), oil transformed, oil moving average
general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)

#### 2-6. Store/Family Specific Feature: Promotion 

In [7]:
df_promotion = pd.concat([df_train, df_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
df_promotion.tail()

## Create promotion time series object by family and store number
family_promotion_dict = {}

for family in family_list:
    df_family = df_promotion.loc[df_promotion['family'] == family]

    list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],
                                value_cols="onpromotion",
                                fill_missing_dates=True,
                                freq='D')
  
    for ts in list_of_TS_promo:
        ts = ts.astype(np.float32)

    family_promotion_dict[family] = list_of_TS_promo

## Transform promotion time series
promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
    promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
    promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

    promo_pipeline = Pipeline([promo_filler,
                             promo_scaler])
  
    promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])
    promotion_transformed_dict[key] = promotion_transformed

## Moving Averages for Promotion Family Dictionaries
promo_moving_average_7 = MovingAverageFilter(window=7)
promo_moving_average_28 = MovingAverageFilter(window=28)

promotion_covs = []

for ts in promotion_transformed:
    ma_7 = promo_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())  
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
        
    ma_28 = promo_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())  
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
    promo_and_mas = ts.stack(ma_7).stack(ma_28)
    promotion_covs.append(promo_and_mas)

    promotion_transformed_dict[key] = promotion_covs

#-- [dict] family_promotion_dict, promotion_transformed_dict

100%|██████████| 33/33 [00:15<00:00,  2.20it/s]


#### 2-7. Store Specific Featue I: Transactions

In [9]:
df_transactions.sort_values(["store_nbr","date"], inplace=True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                df_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],  # individual time series are extracted by grouping `df` by `group_cols`
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())   # necessary workaround to remove static covariates (so I can stack covariates later on)
            series = series.astype(np.float32)
            transactions_list.append(series)

transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

from datetime import datetime, timedelta

transactions_list_full = []

for ts in transactions_list:
  if ts.start_time() > pd.Timestamp('20130101'):
    end_time = (ts.start_time() - timedelta(days=1))
    delta = end_time - pd.Timestamp('20130101')
    zero_series = TimeSeries.from_times_and_values(
                              times=pd.date_range(start=pd.Timestamp('20130101'), 
                              end=end_time, freq="D"),
                              values=np.zeros(delta.days+1))
    ts = zero_series.append(ts)
    transactions_list_full.append(ts)

transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

# Moving Averages for Transactions
trans_moving_average_7 = MovingAverageFilter(window=7)
trans_moving_average_28 = MovingAverageFilter(window=28)

transactions_covs = []

for ts in transactions_transformed:
  ma_7 = trans_moving_average_7.filter(ts).astype(np.float32)
  ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="transactions_ma_7")
  ma_28 = trans_moving_average_28.filter(ts).astype(np.float32)
  ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="transactions_ma_28")
  trans_and_mas = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions").stack(ma_7).stack(ma_28)
  transactions_covs.append(trans_and_mas)


#### 2-8. Store Specific Feature II: Holidays

In [10]:
def create_clean_holiday_data(df_holidays_events, store_info):
    df_holiday = df_holidays_events.copy()
    df_holiday['local_holiday'] = np.where(((df_holiday["type"] == "Holiday") & ((df_holiday["locale_name"] == store_info['state']) | (df_holiday["locale_name"] == store_info['city']))), 1, 0)
    df_holiday = df_holiday.assign(
        national_holiday=np.where((df_holiday["type"] == "Holiday") & (df_holiday["locale"] == "National"), 1, 0),
        earthquake_relief=np.where(df_holiday['description'].str.contains('Terremoto Manabi'), 1, 0),
        christmas=np.where(df_holiday['description'].str.contains('Navidad'), 1, 0),
        football_event=np.where(df_holiday['description'].str.contains('futbol'), 1, 0),
        national_event=np.where((df_holiday["type"] == "Event") & (df_holiday["locale"] == "National"), 1, 0),
        work_day=np.where((df_holiday["type"] == "Work Day"), 1, 0)
    ).drop(['type', 'locale', 'locale_name', 'description'], axis=1)

    # Filter out rows with all zeros and then set 'date' as index
    df_holiday = df_holiday[~(df_holiday.drop('date', axis=1) == 0).all(axis=1)]
    df_holiday = df_holiday.set_index('date')

    # Group by date and take the maximum value for each day
    df_holiday = df_holiday.groupby('date').max().reset_index()

    return df_holiday

In [12]:
def convert_to_timeseries(df):
    holiday_ts = TimeSeries.from_dataframe(df, time_col='date', fill_missing_dates=True, fillna_value=0, freq='D')
    holiday_ts = holiday_ts.slice(pd.Timestamp('20130101'), pd.Timestamp('20170831'))
    holiday_ts = holiday_ts.astype(np.float32)
    return holiday_ts

In [13]:
def process_and_stack_covariates(holiday_ts, general_covariates, transactions_covs, pipeline):
    # Preprocess the holiday timeseries
    holiday_ts_processed = pipeline.fit_transform([holiday_ts])[0]

    # Stack future covariates
    stacked_covariates_future = holiday_ts_processed.stack(general_covariates)

    # Stack past covariates
    holiday_ts_processed = holiday_ts_processed.slice_intersect(transactions_covs)
    general_covariates_sliced = general_covariates.slice_intersect(transactions_covs)
    stacked_covariates_past = transactions_covs.stack(holiday_ts_processed).stack(general_covariates_sliced)

    return stacked_covariates_future, stacked_covariates_past

In [14]:
def prepare_store_covariates(df_stores, df_holidays_events, general_covariates, transactions_covs, pipeline):
    store_covariates_future = []
    store_covariates_past = []

    for i in range(len(df_stores)):
        store_info = df_stores.iloc[i]
        df_holiday = create_clean_holiday_data(df_holidays_events, store_info)
        holiday_ts = convert_to_timeseries(df_holiday)
        stacked_future, stacked_past = process_and_stack_covariates(holiday_ts, general_covariates, transactions_covs[i], pipeline)

        store_covariates_future.append(stacked_future)
        store_covariates_past.append(stacked_past)

    return store_covariates_future, store_covariates_past

In [18]:
holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

pipeline = Pipeline([holidays_filler, holidays_scaler])
store_covariates_future, store_covariates_past = prepare_store_covariates(df_stores, df_holiday, general_covariates, transactions_covs, pipeline)

#### 2-9. Create Past and Future Covariates

In [19]:
## past covariates: sales moving average (store/product level), promotions (store/product level), holiday (store level)
past_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  sales_mas = sales_ma_dict[key]
  covariates_past = [promotion_family[i].slice_intersect(store_covariates_past[i]).stack(store_covariates_past[i].stack(sales_mas[i])) for i in range(0,len(promotion_family))]

  past_covariates_dict[key] = covariates_past


## future covariates: promotions (store/product level), holiday (store level)
future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]

  future_covariates_dict[key] = covariates_future


## only past covariates: sales moving average and transactions
only_past_covariates_dict = {}

for key in tqdm(sales_ma_dict):
  sales_moving_averages = sales_ma_dict[key]
  only_past_covariates = [sales_moving_averages[i].stack(transactions_covs[i]) for i in range(0,len(sales_moving_averages))]

  only_past_covariates_dict[key] = only_past_covariates

100%|██████████| 33/33 [00:02<00:00, 13.26it/s]


In [56]:
# Delete Original Dataframes to Save Memory

del(df_train)
del(df_test)
del(df_stores)
del(df_holiday)
del(df_oil)
del(df_transactions)
gc.collect()


0

### 3. Baseline Time Series Model

#### Exponential Smoothing

In [18]:
# define function to apply exponential smoothing
def apply_exponential_smoothing(ts, model_params = None):
    if model_params is None:
        model_params = {}
    model = ExponentialSmoothing(**model_params)
    model.fit(ts)
    return model.predict(16)

In [28]:
# apply model to each time series

st = time.time()

family_forecasts_dict = {}

for family, ts_list in family_trans_dict.items():
    
    family_forecasts = []

    # apply model for each time series
    for ts in ts_list:
        training_data = ts[:-16]
        forecasts = apply_exponential_smoothing(training_data)
        family_forecasts.append(forecasts)
    
    # transform back    
    family_forecasts_dict[family] = family_pipeline_dict[family].inverse_transform(family_forecasts, partial=True)
    # Zero Forecasting
    for i in range(0,len(family_forecasts_dict[family])):
        if (training_data[i].univariate_values()[-14:] == 0).all():
            family_forecasts_dict[family][i] = family_forecasts_dict[family][i].map(lambda x: x * 0)

# get the execution time
et = time.time()
elapsed_time_exp = et - st

In [30]:
# Function to flatten nested lists
def flatten(l):
    return [item for sublist in l for item in sublist]

# Create lists for actual and predicted values, including only the last 16 values of each series
actual_list = flatten([ts[-16:] for ts in flatten(family_trans_dict.values())])
pred_list_ES = flatten([ts for ts in flatten(family_forecasts_dict.values())])

# Calculate Mean RMSLE for the last 16 values
ES_rmsle = rmsle(actual_series=actual_list,
                 pred_series=pred_list_ES,
                 n_jobs=-1,
                 inter_reduction=np.mean)

print("\n")
print("The mean RMSLE for Exponential Smoothing Models is {:.5f}.".format(ES_rmsle))
print('Training & Inference duration:', elapsed_time_exp, 'seconds')
print("\n")



The mean RMSLE for Exponential Smoothing Models is 1.41878.
Training & Inference duration: 452.0441982746124 seconds




### 4. Boosting (CatBoost, XGBoost)

#### 4-1. CatBoost for 33 Product Families

In [ ]:
catboost_models = {}
catboost_preds = {}

for family in tqdm(family_list):

    # define target, future and past covariates
    sf = family_trans_dict[family]
    future = future_covariates_dict[family]
    past = only_past_covariates_dict[family]

    # train test split
    val_len = 16
    train = [s[:-val_len] for s in sf]
    val = [s[(-val_len +63):] for s in sf]

    # initiate model
    model = CatBoostModel(
        lags=12,
        lags_past_covariates=12,
        lags_future_covariates=[0,1,2,3,4,5],
        output_chunk_length=6)

    # train model for each family
    print('Training model for ...', family)
    model.fit(train, past_covariates= past, future_covariates = future)
    catboost_models[family] = model

    # predict model for each family
    print('Predicting values for ...', family)
    pred = catboost_models[family].predict(n=16,
                                            series = train,
                                            future_covariates = future,
                                            past_covariates = past)
    catboost_preds[family] = pred

In [ ]:
# Transform Back
catboost_back = {}
for family in tqdm(family_list):
  catboost_back[family] = family_pipeline_dict[family].inverse_transform(catboost_back[family], partial=True)

In [ ]:
def flatten(l):
  return [item for sublist in l for item in sublist]

pred_list_cb = flatten([catboost_back[family] for family in family_list])
actual_list = flatten([family_trans_dict[family] for family in family_list])

# Mean RMSLE
cb_rmsle = rmsle(actual_series = actual_list,
                 pred_series = pred_list_cb,
                 n_jobs = -1,
                 inter_reduction=np.mean)

print("\n")
print("The mean RMSLE for the 33 CatBoost Global Product Family Models over all 1782 series is {:.5f}.".format(cb_rmsle))
print("\n")

#### 4-2. XGBoost for 33 Product Families

In [ ]:
from darts.models import XGBModel

xgb_models = {}
xgb_preds = {}

for family in tqdm(family_list):

    # define target, future and past covariates
    sf = family_trans_dict[family]
    future = future_covariates_dict[family]
    past = only_past_covariates_dict[family]

    # train test split
    val_len = 16
    train = [s[:-val_len] for s in sf]
    val = [s[(-val_len +63):] for s in sf]

    # initiate model
    model = XGBModel(
        lags=12,
        lags_past_covariates=12,
        lags_future_covariates=[0,1,2,3,4,5],
        output_chunk_length=6)

    # train model for each family
    print('Training model for ...', family)
    model.fit(train, past_covariates= past, future_covariates = future)
    xgb_models[family] = model

    # predict model for each family
    print('Predicting values for ...', family)
    pred = xgb_models[family].predict(n=16,
                                            series = train,
                                            future_covariates = future,
                                            past_covariates = past)
    xgb_preds[family] = pred

In [ ]:
# Transform Back
xgb_back = {}
for family in tqdm(family_list):
  xgb_back[family] = family_pipeline_dict[family].inverse_transform(xgb_back[family], partial=True)

In [ ]:
def flatten(l):
  return [item for sublist in l for item in sublist]

pred_list_xgb = flatten([xgb_back[family] for family in family_list])
actual_list = flatten([family_trans_dict[family] for family in family_list])

# Mean RMSLE
xgb_rmsle = rmsle(actual_series = actual_list,
                 pred_series = pred_list_xgb,
                 n_jobs = -1,
                 inter_reduction=np.mean)

print("\n")
print("The mean RMSLE for the 33 XGBoost Global Product Family Models over all 1782 series is {:.5f}.".format(xgb_rmsle))
print("\n")

### 5. Deep Learning (LSTM)

In [105]:
# Data Preparation for LSTM

def flatten(l):
  return [item for sublist in l for item in sublist]

future_covariates_full = []

for family in family_list:
  future_covariates_full.append(future_covariates_dict[family])
    
future_covariates_full = flatten(future_covariates_full)

LSTM_covariates = future_covariates_full
    
# Slice-Intersect target and covariates after shifting

LSTM_target = []

for family in family_list:
  LSTM_target.append(family_trans_dict[family])

LSTM_target = flatten(LSTM_target)

# Split in train/val/test for Tuning and Validation

val_len = 16

LSTM_train = [s[: -(2 * val_len)] for s in LSTM_target]
LSTM_val = [s[-(2 * val_len) : -val_len] for s in LSTM_target]
LSTM_test = [s[-val_len:] for s in LSTM_target]

In [184]:
import torch
from darts.models import RNNModel
import time

def build_fit_lstm_model(input_chunk_length, hidden_dim, n_rnn_layers, lr):
    torch.manual_seed(42)

    # Fixed parameters
    MAX_N_EPOCHS = 100
    MAX_SAMPLES_PER_TS = 60
    BATCH_SIZE = 128
    TRAINING_LENGTH = input_chunk_length + val_len - 1

    # Early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.0001, patience=2, verbose=True)
    callbacks = [early_stopper]

    # Trainer configuration
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "gpus": 1,
            "auto_select_gpus": True,
            "callbacks": callbacks,
        }
        num_workers = 2
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        num_workers = 0

    # Model construction
    model = RNNModel(
        model="LSTM",
        input_chunk_length=input_chunk_length,
        hidden_dim=hidden_dim,
        n_rnn_layers=n_rnn_layers,
        dropout=0,
        training_length=TRAINING_LENGTH,
        n_epochs=MAX_N_EPOCHS,
        batch_size=BATCH_SIZE,
        add_encoders=None,
        likelihood=None, 
        loss_fn=torch.nn.MSELoss(),
        random_state=42,
        optimizer_kwargs={"lr": lr},
        pl_trainer_kwargs=pl_trainer_kwargs,
        model_name="lstm_model",
        force_reset=True,
        save_checkpoints=True,
    )

    # Validation set configuration
    LSTM_train = [s[: -(2 * val_len)] for s in LSTM_target]
    model_val_set = [s[-((2 * val_len) + input_chunk_length) : -val_len] for s in LSTM_target]
    
    # Training the model
    model.fit(
        series=LSTM_train,
        val_series=model_val_set,
        max_samples_per_ts=MAX_SAMPLES_PER_TS,
        num_loader_workers=num_workers,
    )

    # Reload the best model
    model = RNNModel.load_from_checkpoint("lstm_model")
    return model

In [ ]:
lstm_params = {'input_chunk_length': 63, 
               'hidden_dim': 39, 
               'n_rnn_layers': 3, 
               'lr': 0.0019971227090605087}

torch.cuda.empty_cache()

# get the start time
st = time.time()

LSTM_Model = build_fit_lstm_model(**lstm_params)

# Generate Forecasts for the Test Data
preds = LSTM_Model.predict(series=LSTM_test, future_covariates=LSTM_covariates, n=val_len)

# Transform Back
forecasts_back = train_pipeline.inverse_transform(preds, partial=True)

# Zero Forecasting
for n in range(0,len(forecasts_back)):
  if (LSTM_target[n][:-16].univariate_values()[-14:] == 0).all():
        forecasts_back[n] = forecasts_back[n].map(lambda x: x * 0)
        

# get the end time
et = time.time()

# get the execution time
elapsed_time_lstm = et - st

# Mean RMSLE
LSTM_rmsle = rmsle(actual_series = list_of_TS,
                 pred_series = forecasts_back,
                 n_jobs = -1,
                 inter_reduction=np.mean)